# Spotify Data Visualization

## 1.1 Importing data and libraries

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#reading a file
df_raw = pd.read_csv('spotify_charts.csv')

## 1.2 Data exploration and optimization stage

In [3]:
#working on the source file is necessary due to its size

df_raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26147953 entries, 0 to 26147952
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   title    object 
 1   rank     int64  
 2   date     object 
 3   artist   object 
 4   url      object 
 5   region   object 
 6   chart    object 
 7   trend    object 
 8   streams  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 13.1 GB


In [4]:
#checking the number of unique values 

for col in df_raw.columns:
    print(f'unique values in {col}: {df_raw[col].nunique()}')

unique values in title: 164758
unique values in rank: 200
unique values in date: 1826
unique values in artist: 96115
unique values in url: 217644
unique values in region: 70
unique values in chart: 2
unique values in trend: 4
unique values in streams: 788013


In [5]:
#changing of data types from object to category/int for elements that have relatively few unique values (in comparison to number of all entries)

df_raw['title'] = df_raw['title'].astype('category')
df_raw['rank'] = df_raw['rank'].astype('int32')
df_raw['date'] = df_raw['date'].astype('category')
df_raw['artist'] = df_raw['artist'].astype('category')
df_raw['url'] = df_raw['url'].astype('category')
df_raw['region'] = df_raw['region'].astype('category')
df_raw['chart'] = df_raw['chart'].astype('category')
df_raw['trend'] = df_raw['trend'].astype('category')

In [14]:
#checking the number of NaN values 

for col in df_raw.columns:
    print(f'NaN values in {col}: {df_raw[col].isna().sum()}')
    

NaN values in title: 0
NaN values in rank: 0
NaN values in date: 0
NaN values in artist: 0
NaN values in url: 0
NaN values in region: 0
NaN values in chart: 0
NaN values in trend: 0
NaN values in streams: 0


In [8]:
title_is_nan = df_raw['title'].isna()
artist_is_nan = df_raw['artist'].isna()
region_is_nan = df_raw['region'].isna()
chart_is_nan = df_raw['chart'].isna()
trend_is_nan = df_raw['trend'].isna()
streams_is_nan = df_raw['streams'].isna()


In [9]:
# NaN data exploratiom
df_raw[streams_is_nan].head()

,title,rank,date,artist,url,region,chart,trend,streams
12808,Boys,13,2017-08-01,Charli XCX,https://open.spotify.com/track/5o4yGlG0PfeVUa6...,Latvia,viral50,MOVE_UP,NaN
12965,Hold Me,19,2017-08-01,R3HAB,https://open.spotify.com/track/2a9xGmpgNyNz9QV...,Luxembourg,viral50,MOVE_DOWN,NaN
13781,Hola Hola,40,2017-08-01,KARD,https://open.spotify.com/track/3VC1IEz9M1txlMS...,Peru,viral50,NEW_ENTRY,NaN
14042,Cuídate,4,2017-08-01,Zetazen,https://open.spotify.com/track/0WgnbqlHieZmdYa...,Spain,viral50,SAME_POSITION,NaN
14092,Boys,13,2017-08-01,Charli XCX,https://open.spotify.com/track/5o4yGlG0PfeVUa6...,Taiwan,viral50,MOVE_DOWN,NaN


In [10]:
# checking if 'streams' contains any '0' values

streams_is_null = df_raw['streams'] == 0

len(df_raw[streams_is_null])



0

In [11]:
# replacing NaN values with a zeros without losing any data

df_raw['streams'].fillna(value=0, inplace=True)

In [12]:
#changing of data types of 'streams' from float64 to int32

df_raw['streams'] = df_raw['streams'].astype('int32')

In [13]:
# droping rest records containing NaN values
df_raw.dropna(inplace=True)


In [15]:
# checking if there is a memory usage saving
df_raw.info(memory_usage='deep')



# 13.1 GB to 880.0 MB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 26147923 entries, 0 to 26147951
Data columns (total 9 columns):
 #   Column   Dtype   
---  ------   -----   
 0   title    category
 1   rank     int32   
 2   date     category
 3   artist   category
 4   url      category
 5   region   category
 6   chart    category
 7   trend    category
 8   streams  int32   
dtypes: category(7), int32(2)
memory usage: 880.0 MB
